In [1]:
import layoutparser as lp
import cv2
from PIL import Image 
import pytesseract
from bs4 import BeautifulSoup
from doctr.io import DocumentFile
from doctr.models import ocr_predictor

2023-06-13 09:11:04.204635: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/dhruv/anaconda3/lib/python3.9/site-packages/cv2/../../lib64:
2023-06-13 09:11:04.204653: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home/dhruv/anaconda3/lib/python3.9/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [65]:
from xml.etree import ElementTree as ET
from xml.etree.ElementTree import Element as ETElement
from xml.etree.ElementTree import SubElement

In [58]:
imgfile = 'images/p-06.jpg'
ocr_lang = 'Tamil'
model = ocr_predictor(pretrained=True)
doc = DocumentFile.from_images(imgfile)
result = model(doc)
img = cv2.imread(imgfile)

In [87]:
def export_as_xml(self, file_title: str = "docTR - XML export (hOCR)"):
        """Export the page as XML (hOCR-format)
        convention: https://github.com/kba/hocr-spec/blob/master/1.2/spec.md

        Args:
            file_title: the title of the XML file

        Returns:
            a tuple of the XML byte string, and its ElementTree
        """
        p_idx = 1
        block_count: int = 1
        line_count: int = 1
        word_count: int = 1
        height, width = self.dimensions
        language = self.language if "language" in self.language.keys() else "en"
        # Create the XML root element
        page_hocr = ETElement("html", attrib={"xmlns": "http://www.w3.org/1999/xhtml", "xml:lang": str(language)})
        # Create the header / SubElements of the root element
        head = SubElement(page_hocr, "head")
        SubElement(head, "title").text = file_title
        SubElement(head, "meta", attrib={"http-equiv": "Content-Type", "content": "text/html; charset=utf-8"})
        SubElement(
            head,
            "meta",
            attrib={"name": "ocr-system", "content": "python-doctr"},  # type: ignore[attr-defined]
        )
        SubElement(
            head,
            "meta",
            attrib={"name": "ocr-capabilities", "content": "ocr_page ocr_carea ocr_par ocr_line ocrx_word"},
        )
        # Create the body
        body = SubElement(page_hocr, "body")
        SubElement(
            body,
            "div",
            attrib={
                "class": "ocr_page",
                "id": f"page_{p_idx + 1}",
                "title": f"image; bbox 0 0 {width} {height}; ppageno 0",
            },
        )
        # iterate over the blocks / lines / words and create the XML elements in body line by line with the attributes
        for block in self.blocks:
            if len(block.geometry) != 2:
                raise TypeError("XML export is only available for straight bounding boxes for now.")
            (xmin, ymin), (xmax, ymax) = block.geometry
            block_div = SubElement(
                body,
                "div",
                attrib={
                    "class": "ocr_carea",
                    "id": f"block_{block_count}",
                    "title": f"bbox {int(round(xmin * width))} {int(round(ymin * height))} \
                    {int(round(xmax * width))} {int(round(ymax * height))}",
                },
            )
            paragraph = SubElement(
                block_div,
                "p",
                attrib={
                    "class": "ocr_par",
                    "id": f"par_{block_count}",
                    "title": f"bbox {int(round(xmin * width))} {int(round(ymin * height))} \
                    {int(round(xmax * width))} {int(round(ymax * height))}",
                },
            )
            block_count += 1
            for line in block.lines:
                (xmin, ymin), (xmax, ymax) = line.geometry
                # NOTE: baseline, x_size, x_descenders, x_ascenders is currently initalized to 0
                line_span = SubElement(
                    paragraph,
                    "span",
                    attrib={
                        "class": "ocr_line",
                        "id": f"line_{line_count}",
                        "title": f"bbox {int(round(xmin * width))} {int(round(ymin * height))} \
                        {int(round(xmax * width))} {int(round(ymax * height))}; \
                        baseline 0 0; x_size 0; x_descenders 0; x_ascenders 0",
                    },
                )
                line_count += 1
                for word in line.words:
                    (xmin, ymin), (xmax, ymax) = word.geometry
                    conf = word.confidence
                    word_div = SubElement(
                        line_span,
                        "span",
                        attrib={
                            "class": "ocrx_word",
                            "id": f"word_{word_count}",
                            "title": f"bbox {int(round(xmin * width))} {int(round(ymin * height))} \
                            {int(round(xmax * width))} {int(round(ymax * height))}; \
                            x_wconf {int(round(conf * 100))}",
                        },
                    )
                    # set the text
                    word_div.text = word.value
                    word_count += 1

        return ET.tostring(page_hocr, encoding="unicode", method="xml")

In [68]:
# Page image read
img = cv2.imread(imgfile)
for page in result.pages:
    for block in page.blocks:
        for line in block.lines:
            for word in line.words:
                ((x1,y1),(x2,y2)) = word.geometry
                x1 = int(x1 * width)
                x2 = int(x2 * width)
                y1 = int(y1 * height)
                y2 = int(y2 * height)
                bb = ((x1,y1),(x2,y2))
                #Crop the block
                cropped_image = img[bb[0][1]:bb[1][1], bb[0][0]:bb[1][0]]
                #Perform OCR
                text = pytesseract.image_to_string(cropped_image, lang=ocr_lang)
                word.value = text

In [69]:
page_result = result.pages[0]

In [89]:
hocr_file_content = export_as_xml(page_result)

In [90]:
f = open('try.html', 'w+')
f.write(hocr_file_content)
f.close()

### Just OLD Experimentation

In [46]:
jsonresult['pages'][0]['blocks'][2]['lines'][0]['words']

[{'value': '5TA15606T',
  'confidence': 0.017533766105771065,
  'geometry': ((0.1474609375, 0.1552734375), (0.240234375, 0.173828125))},
 {'value': 'FDSUuSG',
  'confidence': 0.001590757630765438,
  'geometry': ((0.24609375, 0.154296875), (0.357421875, 0.1748046875))},
 {'value': 'LOl564L',
  'confidence': 0.03302145376801491,
  'geometry': ((0.365234375, 0.1552734375), (0.439453125, 0.1728515625))},
 {'value': 'LDAILD4A',
  'confidence': 0.002667727181687951,
  'geometry': ((0.447265625, 0.1552734375), (0.5302734375, 0.173828125))},
 {'value': '9160LGICND6OT.',
  'confidence': 0.0031700306572020054,
  'geometry': ((0.5390625, 0.1552734375), (0.669921875, 0.1728515625))}]

In [3]:
#Export result
jsonresult = result.export()
height, width = jsonresult['pages'][0]['dimensions']

In [14]:
jsonresult

{'pages': [{'page_idx': 0,
   'dimensions': (3509, 2480),
   'orientation': {'value': None, 'confidence': None},
   'language': {'value': None, 'confidence': None},
   'blocks': [{'geometry': ((0.1474609375, 0.0888671875),
      (0.2158203125, 0.1064453125)),
     'lines': [{'geometry': ((0.1474609375, 0.0888671875),
        (0.2158203125, 0.1064453125)),
       'words': [{'value': 'ஜயா\n\x0c',
         'confidence': 0.22553536295890808,
         'geometry': ((0.1474609375, 0.0888671875),
          (0.2021484375, 0.1064453125))},
        {'value': '\x0c',
         'confidence': 0.9988914728164673,
         'geometry': ((0.2060546875, 0.0908203125),
          (0.2158203125, 0.103515625))}]}],
     'artefacts': []},
    {'geometry': ((0.20703125, 0.119140625), (0.87890625, 0.140625)),
     'lines': [{'geometry': ((0.20703125, 0.119140625),
        (0.87890625, 0.140625)),
       'words': [{'value': 'வணக்கம்\u200c,\n\x0c',
         'confidence': 0.17531739175319672,
         'geometry': (

In [49]:
type(hocr_file_content[0])

tuple

In [13]:
bboxes = []
for block in jsonresult['pages'][0]['blocks']:
    for lines in block['lines']:
        for words in lines['words']:
            ((x1,y1),(x2,y2)) = words['geometry']
            x1 = int(x1 * width)
            x2 = int(x2 * width)
            y1 = int(y1 * height)
            y2 = int(y2 * height)
            bb = ((x1,y1),(x2,y2))
            #Crop the block
            cropped_image = img[bb[0][1]:bb[1][1], bb[0][0]:bb[1][0]]
            #Perform OCR
            text = pytesseract.image_to_string(cropped_image, lang=ocr_lang)
            words['value'] = text
            print(words['value'])

ஜயா


வணக்கம்‌,


சமாஜ்‌

கட்சி

சார்பாக

இக்கடித்த்தின்‌

மூலம்‌

தங்களை

சந்திப்பதில்‌

மிகவும்‌

மகிழ்ச்சி

அடைகிறேன்‌.

நான்‌

மேற்கண்ட

முகவரியில்‌

வசித்து

ன்‌.
வருகிறே

நான்‌

தாழ்த்தப்பட்ட

இந்து

வள்ளுவன்‌

இனத்தைச்‌

சார்ந்தவன்‌.

திருவண்ணாமலை

டவுன்‌


வது

வார்டு

32


பிளாக்‌

சகதுளிமலை

சர்வே


1826/2A,

1826/2B,

1826/2C

ஆகியவற்றில்‌

உள்ள

பஞ்சமி

நிலம்‌


ஏக்கர்‌


சென்ட்‌

பஞ்சமி

நிலம்‌

கடந்த

(30.11.1924)-ல்‌

B.K.N

1706/32

அரசால்‌

இலவசமாக

மேற்படி

திருவண்ணாமலை

டவுன்‌

சமுத்திரம்‌

வள்ளுவர்‌

தெருவில்‌

வசித்த

(Late)

திரு.

சுப்புராய

பண்டாரம்‌

துபெ

திரு.

நல்லமுத்து

என்பவருக்கு

பஞ்சமி

நிலம்‌

அரசாங்கத்தால்‌

வழங்கப்பட்டது.

நிலமானது

தொடர்ந்து

பயன்பாட்டில

இருந்துள்ளது.

இந்நிலையில்‌

ஒரு

சில

ஆதிக்க

வர்க்கத்தினர்‌

ஆக்கிரமிப்பால்‌

1990-ல்‌


Re

Rolling

Mills

மேற்படி

சர்வே



சுமார்‌

0.95

சென்ட்‌

பரப்பளவில்‌

துவங்கப்பட்டுள்ளது.

இதனை


சென்

In [42]:
f = open("try.html", 'w+')
f.write(str(hocr_file_content))
f.close()

In [33]:
# blockbboxes = []
# for block in jsonresult['pages'][0]['blocks']:
#     ((x1,y1),(x2,y2)) = block['geometry']
#     x1 = int(x1 * width)
#     x2 = int(x2 * width)
#     y1 = int(y1 * height)
#     y2 = int(y2 * height)
#     bbox = ((x1,y1),(x2,y2))
#     print(bbox)
#     blockbboxes.append(bbox)

((365, 311), (535, 373))
((513, 418), (2179, 493))
((365, 541), (1661, 613))
((506, 654), (968, 729))
((1058, 657), (2177, 729))
((368, 777), (695, 839))
((835, 771), (971, 846))
((1104, 777), (1365, 839))
((1501, 774), (1765, 846))
((1903, 777), (2174, 839))
((360, 884), (2179, 1648))
((360, 1689), (2177, 2330))
((363, 2374), (891, 2436))
((506, 2480), (2167, 2559))
((356, 2604), (748, 2789))
((857, 2597), (1060, 2659))
((1072, 2600), (1259, 2779))
((1324, 2600), (1596, 2768))
((1600, 2604), (1726, 2655))
((1702, 2604), (2167, 2779))
((578, 2717), (963, 2779))
((363, 2827), (2170, 2902))
((360, 2943), (775, 3015))
((864, 2947), (1060, 2998))
((1147, 2947), (1375, 2998))
((1467, 2947), (1925, 3008))
((2017, 2947), (2167, 3012))
((365, 3053), (2172, 3128))


In [11]:
bboxes = []
for block in jsonresult['pages'][0]['blocks']:
    for lines in block['lines']:
        for words in lines['words']:
            print(words)
            ((x1,y1),(x2,y2)) = words['geometry']
            x1 = int(x1 * width)
            x2 = int(x2 * width)
            y1 = int(y1 * height)
            y2 = int(y2 * height)
            bbox = ((x1,y1),(x2,y2))
            print(bbox)
            bboxes.append(bbox)

{'value': 'Qumr', 'confidence': 0.22553536295890808, 'geometry': ((0.1474609375, 0.0888671875), (0.2021484375, 0.1064453125))}
((365, 311), (501, 373))
{'value': ':', 'confidence': 0.9988914728164673, 'geometry': ((0.2060546875, 0.0908203125), (0.2158203125, 0.103515625))}
((511, 318), (535, 363))
{'value': '621600165L0,', 'confidence': 0.17531739175319672, 'geometry': ((0.20703125, 0.12109375), (0.306640625, 0.138671875))}
((513, 424), (760, 486))
{'value': 'L(5260T', 'confidence': 0.1510981321334839, 'geometry': ((0.32421875, 0.12109375), (0.4013671875, 0.1396484375))}
((804, 424), (995, 490))
{'value': 'FLOITO', 'confidence': 0.22988136112689972, 'geometry': ((0.416015625, 0.12109375), (0.48046875, 0.1396484375))}
((1031, 424), (1191, 490))
{'value': 'BLA', 'confidence': 0.7943438291549683, 'geometry': ((0.4951171875, 0.1201171875), (0.5498046875, 0.138671875))}
((1227, 421), (1363, 486))
{'value': 'FITIJLITS', 'confidence': 0.2636241316795349, 'geometry': ((0.56640625, 0.1220703125

In [53]:
# Saving blockwise results if needed
img = cv2.imread(imgfile)
for b in bboxes:
    cv2.rectangle(img, b[0], b[1], (255, 255, 25), 4)
cv2.imwrite(imgfile[:-4] + '_word_level.jpg', img)

True

In [54]:
hocr_start_template = f'<?xml version="1.0" encoding="UTF-8"?><!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN""http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd"><html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml"><head><title></title><meta content="text/html;charset=utf-8" http-equiv="Content-Type"/><meta content="tesseract 4.1.1" name="ocr-system"/><meta content="ocr_page ocr_carea ocr_par ocr_line ocrx_word ocrp_wconf" name="ocr-capabilities"/></head><body><div class="ocr_page" id="page_1" title=\'image "{imgfile}"; bbox 0 0 {width} {height}; ppageno 0\'>'

In [55]:
hocr_end_template = '</div></body></html>'

In [56]:
def get_hocr_block_template(ocr_lang, text, bbox):
    temp = f'<p class="ocr_par" id="par_1_1" lang="{ocr_lang}" title="bbox {bbox[0][0]} {bbox[0][1]} {bbox[1][0]} {bbox[1][1]}">{text}</p>'
    return temp

In [57]:
final_text = ''
final_hocr = hocr_start_template
# Read fresh image
img = cv2.imread(imgfile)

#Get text blockwise from tesseract
for bb in bboxes:
    #Crop the block
    cropped_image = img[bb[0][1]:bb[1][1], bb[0][0]:bb[1][0]]
    #Perform OCR
    text = pytesseract.image_to_string(cropped_image, lang=ocr_lang)
    final_text = final_text + ' ' + text
    final_hocr = final_hocr + '\n' + get_hocr_block_template(ocr_lang, text, bb)

final_hocr = final_hocr + '\n' + hocr_end_template

In [58]:
with open(imgfile[:-3] + 'txt', 'w+') as f:
    f.write(final_text)
    f.close()

In [60]:
soup = BeautifulSoup(final_hocr, 'html.parser')
hocrfile = imgfile[:-3] + 'html'
hocrfile = open(hocrfile, 'w+')
hocrfile.write(str(soup))
hocrfile.close()

In [61]:
skel_img = img * 0
words = sorted(bboxes, key=lambda b:b[0][1]+b[1][1], reverse=False)

In [62]:
words

[((511, 318), (535, 363)),
 ((365, 311), (501, 373)),
 ((1227, 421), (1363, 486)),
 ((513, 424), (760, 486)),
 ((2012, 418), (2179, 493)),
 ((804, 424), (995, 490)),
 ((1031, 424), (1191, 490)),
 ((1404, 428), (1591, 490)),
 ((1639, 428), (1968, 490)),
 ((905, 544), (1089, 606)),
 ((1336, 544), (1661, 606)),
 ((365, 544), (595, 609)),
 ((1109, 544), (1315, 609)),
 ((610, 541), (886, 613)),
 ((506, 654), (634, 729)),
 ((712, 661), (968, 723)),
 ((1058, 661), (1348, 723)),
 ((1688, 657), (1981, 729)),
 ((1431, 661), (1608, 726)),
 ((2053, 661), (2177, 726)),
 ((368, 777), (695, 839)),
 ((1104, 777), (1365, 839)),
 ((1903, 777), (2174, 839)),
 ((835, 771), (971, 846)),
 ((1501, 774), (1765, 846)),
 ((1048, 884), (1094, 949)),
 ((1690, 887), (1855, 949)),
 ((1445, 890), (1513, 949)),
 ((368, 890), (823, 952)),
 ((864, 890), (1017, 956)),
 ((1256, 890), (1409, 956)),
 ((1896, 897), (2172, 949)),
 ((1114, 894), (1220, 959)),
 ((1550, 897), (1656, 962)),
 ((557, 1004), (670, 1058)),
 ((1196, 